In [1]:
!pip install langchain
!pip install langchain_huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 923.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 11.6 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
import torch
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    max_new_tokens=128,
)

visited_urls = set()

visited_lock = threading.Lock()
file_lock = threading.Lock()
counter_lock = threading.Lock()

processed_links = 0
MAX_LINKS = 100

def process_text_with_llm(text, url):
    """
    Process the scraped text using the language model.
    """
    messages = [
        {"role": "system", "content": "You are a language model that cleans up and summarizes text without losing important information."},
        {"role": "user", "content": text},
    ]

    try:
        output = pipe(messages[1]["content"], max_new_tokens=256)
        cleaned_text = output[0]["generated_text"]

        with file_lock:
            with open('processed_text.txt', 'a', encoding='utf-8') as f:
                f.write(f"{cleaned_text.strip()}\n")
                f.write(f"{'-'*80}\n\n")

    except torch.cuda.OutOfMemoryError:
        torch.cuda.empty_cache()
        print(f"Out of memory when processing URL: {url}")
    except Exception as e:
        print(f"Error during LLM processing: {e}")

def scrape_text(url, max_depth=3, depth=0):
    """
    Recursively scrape text from the given URL and its child links.
    """
    global processed_links

    with counter_lock:
        if processed_links >= MAX_LINKS:
            return

    with visited_lock:
        if url in visited_urls or depth > max_depth:
            return
        visited_urls.add(url)

    print(f"Processing link {processed_links+1}/{MAX_LINKS}: {url}")

    try:
        headers = {'User-Agent': 'Mozilla/5.0 (compatible; WebScraper/1.0)'}
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        encoding = response.encoding or 'utf-8'
        page_content = response.content.decode(encoding, errors='replace')

        soup = BeautifulSoup(page_content, 'html.parser')

    except requests.RequestException as e:
        print(f"Failed to retrieve {url}: {e}")
        return
    except UnicodeDecodeError as e:
        print(f"Encoding error at {url}: {e}")
        return

    page_text = soup.get_text(separator=' ', strip=True)
    page_text = page_text.encode('ascii', 'ignore').decode('utf-8')

    with counter_lock:
        processed_links += 1

    process_text_with_llm(page_text, url)
    with counter_lock:
        if processed_links >= MAX_LINKS:
            return

    links_to_scrape = []
    for link_tag in soup.find_all('a', href=True):
        href = link_tag['href']
        child_url = urljoin(url, href)

        if urlparse(child_url).netloc != urlparse(url).netloc:
            continue

        with counter_lock:
            if processed_links >= MAX_LINKS:
                break

        links_to_scrape.append(child_url)

    return links_to_scrape

def main(starting_url):
    with ThreadPoolExecutor(max_workers=3) as executor:
        futures = {executor.submit(scrape_text, starting_url): starting_url}

        while futures and processed_links < MAX_LINKS:
            for future in as_completed(futures):
                try:
                    new_links = future.result()
                    if new_links:
                        for link in new_links:
                            if processed_links >= MAX_LINKS:
                                break
                            futures[executor.submit(scrape_text, link)] = link

                except Exception as exc:
                    print(f"Error during scraping: {exc}")

                del futures[future]

if __name__ == "__main__":
    starting_url = input("Enter the URL to scrape: ")
    with open('processed_text.txt', 'w', encoding='utf-8') as f:
        f.write('')
    main(starting_url)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing link 99/100: https://www.cmu.edu/leadership/the-provost/academic-leadership/culyba-bio.html


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Processing link 100/100: https://www.cmu.edu/education-office/meet-the-vp/index.html


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
